In [1]:
from py_ecc.bn128 import G1, multiply, add, FQ, eq, Z1, curve_order
from py_ecc.bn128 import curve_order as p
import numpy as np
from functools import reduce
import random
import galois

import sys
sys.path.append('../hw_1')
from pedersen_commitment import generate_n_random_points, pedersen_commitment

In [2]:
GF = galois.GF(curve_order)

In [3]:
def gf_inverse(a):
    return GF(1) / a

In [21]:
def random_element():
    return random.randint(0, 1000)

def add_points(points):
    return reduce(add, points, Z1)

# if points = G1, G2, G3, G4 and scalars = a,b,c,d vector_commit returns
# aG1 + bG2 + cG3 + dG4
def vector_commit(points, scalars):
    # return reduce(add, [multiply(P, i) for P, i in zip(points, scalars)], Z1)
    result = Z1
    for P, i in zip(points, scalars):
        result = add(result, multiply(P, int(i)))
    return result


In [22]:
G_vec = generate_n_random_points('hello', 2)

In [23]:
# return a folded vector of length n/2 for scalars
def fold(scalar_vec, u):
    # if scalar_vec is odd add 0 to the end
    if len(scalar_vec) % 2 == 1:
        scalar_vec.append(0)
    # fold the vector
    return [scalar_vec[i] * u + gf_inverse(u) * scalar_vec[i + 1] for i in range(0, len(scalar_vec), 2)]


In [24]:
# return a folded vector of length n/2 for points
def fold_points(point_vec, u):
    if len(point_vec) % 2 == 1:
        point_vec.append(Z1)
    # fold the vector
    return [add(multiply(point_vec[i], int(u)), multiply(point_vec[i + 1], int(gf_inverse(u)))) for i in range(0, len(point_vec), 2)]


In [25]:
def compute_secondary_diagonal(G_vec, a):
    if len(a) % 2 == 1:
        a.append(0)
        G_vec.append(Z1)
    L = vector_commit([G_vec[i+1] for i in range(0, len(G_vec)-1, 2)], [a[i] for i in range(0, len(a)-1, 2)])
    R = vector_commit([G_vec[i] for i in range(0, len(G_vec)-1, 2)], [a[i+1] for i in range(0, len(a)-1, 2)])

    return L, R

In [38]:
def compute_secondary_diagonal_scalar(b, a):
    R = 0
    L = 0

    for i in range(len(a)):
        if i % 2 == 0:
            R = (R + (b[i] * a[i+1]))
        else:
            L = (L + (b[i] * a[i-1]))

    return L, R

In [40]:
a = [GF(1), GF(2)]
b = [GF(5), GF(6)]

g_vec = generate_n_random_points('hello', 4)
h_vec = generate_n_random_points('bullet', 4)

q = generate_n_random_points('q', 1)[0]

q_vec = []
for i in range(len(b)):
    q_vec.append(multiply(q, int(b[i])))

p = add(add(vector_commit(g_vec, a), vector_commit(h_vec, b)), vector_commit(q_vec, a))

l_1, r_1 = compute_secondary_diagonal(g_vec, a)
l_2, r_2 = compute_secondary_diagonal(h_vec, b)
l_3, r_3 = compute_secondary_diagonal(q_vec, a)

l = add(add(l_1, l_2), l_3)
r = add(add(r_1, r_2), r_3)

print(l)
print(r)

u = GF(random_element())

u_squared = u * u
u_inv_squared = gf_inverse(u) * gf_inverse(u)
lu_2 = multiply(l, int(u_squared))
ru_2 = multiply(r, int(u_inv_squared))
p_prime = add(add(lu_2, p), ru_2)

g_prime = fold_points(g_vec, gf_inverse(u))
h_prime = fold_points(h_vec, u)

a_prime = fold(a, u)
b_prime = fold(b, gf_inverse(u))

q_prime = []
for i in range(len(b_prime)):
    q_prime.append(multiply(q, int(b_prime[i])))
  

p_1 = add(add(vector_commit(g_prime, a_prime), vector_commit(h_prime, b_prime)), vector_commit(q_prime, a_prime))

verification = eq(p_1, add(add(vector_commit(g_prime, a_prime), vector_commit(h_prime, b_prime)), multiply(q, int(a_prime[0] * b_prime[0]))))

print(verification)



(16190011361069560923400258561835911105270653408153371243694959837297232081176, 18742250086670805958315119482117758118928440492373928207378521063705862837865)
(7726591647306897651843342549465409067748489424723756381997609426279776040689, 8481583831227148534844669596656539200368725732236479184021083856547444881773)
True
